# Low-Rank Adaptation (LoRA)

Low-Rank Adaptation (LoRA) is a technique designed to efficiently adapt large machine learning models to specific tasks or domains without the need for full retraining. This method introduces lightweight, trainable components to the existing model, significantly reducing computational and memory requirements.

In [1]:
%%capture
import sys

# Añade el directorio principal al path de búsqueda para importar módulos desde esa ubicación
sys.path.insert(0, "..")

# Desactivar los warnings para evitar mensajes innecesarios durante la ejecución
import warnings

# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

from likelihood.models.deep import (
    AutoClassifier,
)  # Modelos de deep learning personalizados
from likelihood.tools import OneHotEncoder, get_metrics, apply_lora  # Herramientas auxiliares

import tensorflow as tf

is_updated = False
from packaging import version

if version.parse(tf.__version__) > version.parse("2.15.0"):
    is_updated = True

E0000 00:00:1755452250.801737   16728 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755452250.806043   16728 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755452250.817729   16728 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755452250.817743   16728 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755452250.817745   16728 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755452250.817746   16728 computation_placer.cc:177] computation placer already registered. Please check linka

## Key Concepts

- **Model Adaptation**: LoRA modifies pre-trained models by adding small, trainable matrices to each layer, allowing the model to adjust to new tasks with minimal additional training.

- **Low-Rank Matrices**: The technique leverages low-rank matrices, which are smaller and require less memory and computational power, to efficiently adapt the model.

## Benefits Over Full Fine-Tuning

- **Reduced Training Costs**: LoRA requires fewer resources compared to full fine-tuning, making it accessible for teams with limited computational power.

- **Maintained Performance**: Despite having fewer trainable parameters, models adapted with LoRA can achieve performance levels comparable to those fine-tuned fully.

For a more in-depth understanding of LoRA, consider the following resources:

- **Original Research Paper**: ["LoRA: Low-Rank Adaptation of Large Language Models"](https://arxiv.org/abs/2106.09685)

- **GitHub Repository**: [Microsoft's LoRA Implementation](https://github.com/microsoft/LoRA)

- **Hugging Face Documentation**: [LoRA in Transformers](https://huggingface.co/docs/diffusers/v0.21.0/en/training/lora)

- **Comprehensive Survey**: [Awesome-LoRA-Low-Rank-Adaptation](https://github.com/lliai/Awesome-LoRA-Low-Rank-Adaptation)

In [2]:
# Cargar el dataset de cáncer de mama desde sklearn
df = datasets.load_breast_cancer()

# Convertir los datos a un DataFrame de pandas para facilitar la manipulación
df_cancer = pd.DataFrame(data=df.data, columns=df.feature_names)
df_cancer["target"] = df.target  # Añadir la columna de etiquetas 'target'

# OneHotEncoder convierte las etiquetas a formato one-hot encoding
y_encoder = OneHotEncoder()
y = y_encoder.encode(df_cancer["target"].to_list())  # Codificar las etiquetas de la clase (target)
X = df_cancer.drop(
    columns="target"
).to_numpy()  # Extraer las características (sin la columna 'target')
X = np.asarray(X).astype(np.float32)  # Convertir X a tipo float32 para la entrada del modelo
y = np.asarray(y).astype(np.float32)  # Convertir y a tipo float32

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Example usage
input_shape = (X.shape[1],)
num_classes = y.shape[1]

# Define the AutoClassifier model
model = AutoClassifier(
    input_shape_parm=input_shape[-1],
    num_classes=num_classes,
    units=17,
    activation="selu",
    l2_reg=0.0,
    num_layers=2,
    lora_mode=True,
    lora_rank=3,
)

if is_updated:
    model = model._main_model

# Compilación del modelo: optimizador, función de pérdida y métricas
model.compile(
    optimizer="adam",  # Optimizador Adam
    loss=tf.keras.losses.CategoricalCrossentropy(),  # Función de pérdida para clasificación multiclase
    metrics=[
        tf.keras.metrics.F1Score(threshold=0.5)
    ],  # Métrica F1 (threshold = 0.5 para predicciones)
)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=False)

# Hacer predicciones sobre el conjunto de entrenamiento
pred = model.predict(X)

# Convertir las predicciones a las etiquetas predichas (máxima probabilidad)
pred_label = np.argmax(pred, axis=1)

# Añadir las predicciones al DataFrame original para su análisis
df = df_cancer.copy()
y_labels = df.drop(columns="target").columns.to_list()
df_cancer["prediction"] = pred_label  # Columna de las etiquetas predichas
df_cancer["label_0"] = pred[:, 0]  # Probabilidad de la clase 0
df_cancer["label_1"] = pred[:, 1]  # Probabilidad de la clase 1

# Calcular y mostrar las métricas del modelo comparando las etiquetas reales y las predicciones
get_metrics(df_cancer, "target", "prediction", verbose=True)

# Guardar el modelo entrenado en el disco en formato TensorFlow
if is_updated:
    model.save("lora_model.keras")
else:
    model.save("lora_model", save_format="tf")

Input shape: (None, 38)
Dense weights shape: 38x17
LoRA weights shape: A(38, 3), B(3, 17)
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Accuracy: 92.62%
Precision: 89.87%
Recall: 99.44%
F1-Score: 94.41
Cohen's Kappa: 0.8362


In [4]:
# Cargar el modelo guardado desde el disco
if is_updated:
    model = tf.keras.models.load_model("lora_model.keras")
else:
    model = tf.keras.models.load_model("lora_model")
    model.classifier.summary()
# Hacer predicciones nuevamente con el modelo cargado
pred = model.predict(X)

# Obtener las etiquetas predichas para las nuevas predicciones
pred_label = np.argmax(pred, axis=1)

# Añadir las nuevas predicciones al DataFrame original
df["prediction"] = pred_label

# Calcular y mostrar las métricas nuevamente con el modelo cargado
get_metrics(df, "target", "prediction", verbose=True)

Input shape: [None, 38]
Dense weights shape: 38x17
LoRA weights shape: A(38, 3), B(3, 17)
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 92.62%
Precision: 89.87%
Recall: 99.44%
F1-Score: 94.41
Cohen's Kappa: 0.8362
